In [837]:
%load_ext autoreload
%autoreload 2
import npgrad as npg
import torch

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [838]:
x = npg.ndarray([1,2,3,])
y =  npg.ndarray(1)

z = x + y
print(z)

ndarray(data=[2 3 4], grad=None)
